In [1]:
import tensorflow as tf

In [4]:
loaded_model = tf.keras.models.load_model('../model/best_fusion_model(light)_(2025-11-06_18-17).h5')

In [5]:
loaded_model.summary()

Model: "ResNet50_InceptionV3_Fusion"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_image         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet50            │ (None, 7, 7,      │ 23,587,712 │ input_image[0][0] │
│ (Functional)        │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ inception_v3        │ (None, 5, 5,      │ 21,802,784 │ input_image[0][0] │
│ (Functional)        │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet_gap          │ (None, 2048)      │          0 │ resnet50[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ inception_gap       │ (None, 2048)      │          0 │ inception_v3[0][… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ feature_fusion      │ (None, 4096)      │          0 │ resnet_gap[0][0], │
│ (Concatenate)       │                   │            │ inception_gap[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 4096)      │     16,384 │ feature_fusion[0… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fc1 (Dense)         │ (None, 1024)      │  4,195,328 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 1024)      │      4,096 │ fc1[0][0]         │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 1024)      │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fc2 (Dense)         │ (None, 512)       │    524,800 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 512)       │      2,048 │ fc2[0][0]         │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 512)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ predictions (Dense) │ (None, 7)         │      3,591 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 50,136,745 (191.26 MB)

 Trainable params: 35,424,647 (135.13 MB)

 Non-trainable params: 14,712,096 (56.12 MB)

 Optimizer params: 2 (12.00 B)

In [6]:
import json

with open('class_indices.json', 'r') as f:
    class_indices = json.load(f)


In [7]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os

# ======== KONFIGURASI ========
MODEL_PATH = "../model/best_fusion_model(light)_(2025-11-06_18-17).h5"  # ganti dengan path model kamu
IMAGE_PATH = "test.jpg"                      # path gambar yang ingin diprediksi
IMG_SIZE = (224, 224)                                      # sesuaikan dengan input model kamu
# CLASS_NAMES = ['Black Rot', 'Brown Spot', 'Downey Mildew', 'ESCA', 'Healthy', 'Leaf Blight', 'Powdery Mildew']  # ubah sesuai labelmu
CLASS_NAMES = list(class_indices.keys())

# ======== LOAD MODEL ========
model = load_model(MODEL_PATH)
print("✅ Model berhasil dimuat!")

# ======== PREPROCESS GAMBAR ========
# img = load_img(IMAGE_PATH, target_size=IMG_SIZE)
# img_array = img_to_array(img)
# img_array = np.expand_dims(img_array, axis=0)  # ubah jadi batch 1
# img_array = img_array / 255.0                  # normalisasi (jika model dilatih dengan skala ini)

# # ======== INFERENCE ========
# predictions = model.predict(img_array)
# predicted_class = np.argmax(predictions, axis=1)[0]
# confidence = np.max(predictions)

# ======== HASIL ========



✅ Model berhasil dimuat!


In [8]:
def predict(img_path):
    # Preprocess gambar
    img = load_img(img_path, target_size=IMG_SIZE)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # ubah jadi batch 1
    img_array = img_array / 255.0                  # normalisasi (jika model dilatih dengan skala ini)

    # Inference
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)[0]
    confidence = np.max(predictions)

    return CLASS_NAMES[predicted_class], confidence

In [9]:
kelas, conf = predict("esca.jpg")
print(f"🔍 Gambar: {os.path.basename(IMAGE_PATH)}")
print(f"📊 Prediksi kelas: {kelas}")
print(f"💡 Confidence: {conf*100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
🔍 Gambar: test.jpg
📊 Prediksi kelas: ESCA
💡 Confidence: 100.00%


In [16]:
kelas, conf = predict("esca1.jpg")
print(f"🔍 Gambar: {os.path.basename(IMAGE_PATH)}")
print(f"📊 Prediksi kelas: {kelas}")
print(f"💡 Confidence: {conf*100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
🔍 Gambar: test.jpg
📊 Prediksi kelas: ESCA
💡 Confidence: 100.00%


In [18]:
kelas, conf = predict("1.png")
print(f"🔍 Gambar: {os.path.basename(IMAGE_PATH)}")
print(f"📊 Prediksi kelas: {kelas}")
print(f"💡 Confidence: {conf*100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
🔍 Gambar: test.jpg
📊 Prediksi kelas: Healthy
💡 Confidence: 100.00%


'Black Rot'